In [20]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np

def prepare_inputs(text_list, tokenizer):
    return tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")

def average_pool(last_hidden_state, attention_mask):
    masked_hidden_state = last_hidden_state * attention_mask.unsqueeze(-1)
    return masked_hidden_state.sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

def calculate_similarity(embeddings, split_index):
    from torch.nn.functional import cosine_similarity
    output_embeddings = embeddings[:split_index]
    tags_embeddings = embeddings[split_index:]
    similarity_matrix = cosine_similarity(output_embeddings.unsqueeze(1), tags_embeddings.unsqueeze(0), dim=-1)
    best_scores, best_indices = similarity_matrix.max(dim=-1)
    return best_indices, best_scores

def main():
    # トークナイザーとモデルのロード
    tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
    model = AutoModel.from_pretrained("intfloat/multilingual-e5-small")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # データの読み込み
    demo_data1 = pd.read_csv('./data/demo.txt', header=None, names=['text'])  # ①のデータを読み込む
    demo_data2 = pd.read_csv('./data/demo2.txt', header=None, names=['text'])  # ②のデータを読み込む
    
    # テキストリストの結合
    texts = demo_data1['text'].tolist() + demo_data2['text'].tolist()
    
    # トークン化
    inputs = prepare_inputs(texts, tokenizer)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # モデル推論
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 平均プーリング
    embeddings = average_pool(outputs.last_hidden_state, inputs['attention_mask'])
    
    # 埋め込みのCPUへの転送
    embeddings = embeddings.cpu()
    
    # 類似度計算
    split_index = len(demo_data1)
    best_indices, best_scores = calculate_similarity(embeddings, split_index)
    
    # 結果の表示
    results = pd.DataFrame({
        'demo_text': demo_data1['text'],
        'best_match_text': [demo_data2['text'][idx.item()] for idx in best_indices],
        'similarity_score': best_scores.numpy()
    })
    
    print(results)

if __name__ == "__main__":
    main()


                            demo_text                 best_match_text  \
0                こんにちは。チームりんりんのもりりんです        おはようございます。チームりんりんのもりりんです   
1                  私たちの作品は「プレゼンバディです」              私たちの作品は「プレゼンバディです」   
2  プレゼンバディは、プレゼンテーションの発表を手伝ってくれるアプリです              私たちの作品は「プレゼンバディです」   
3      あ、メモりんではなくて、もりりんです。間違えないでくださいね  あ、メモりんではなくて、もりりんです。間違えないでくださいね   

   similarity_score  
0          0.985254  
1          1.000000  
2          0.893822  
3          1.000000  
